## Vector Search

### Embeddings

Qdrant uses fastembed under the hood to turn text into vectors.

We will now explore this library

Make sure it's installed: pip install fastembed

In [71]:
from fastembed import TextEmbedding

### Q1. Embedding the query

Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.
You should get a numpy array of size 512.

What's the minimal value in this array?

In [72]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6334")

In [73]:
client

In [74]:
EMBEDDING_DIMENSIONALITY = 512

In [75]:
query = 'I just discovered the course. Can I join now?'

In [76]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
model = TextEmbedding(model_name=model_handle)

2025-06-14 15:40:34.434 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/models/xenova/jina-embeddings-v2-small-en Falling back to other sources.
2025-06-14 15:40:34.437 | ERROR    | fastembed.common.model_management:download_model:452 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.
2025-06-14 15:40:37.456 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/models/xenova/jina-embeddings-v2-small-en Falling back to other sources.
2025-06-14 15:40:37.460 | ERROR    | fastembed.common.model_management:download_model:452 - Could not download model from either source, sleeping for 9.0 seconds, 1 retries left.
2025-06-14 15:40:46.474 | ERROR    | fastembed.common.model_management:download_mo

ValueError: Could not load model jinaai/jina-embeddings-v2-small-en from any source.

In [9]:
query_embedding = list(model.embed(query))

In [10]:
query_embedding

[array([-7.63946260e-02, -7.30555505e-02,  5.86501763e-02,  3.92670571e-02,
        -1.41797115e-02, -4.68342854e-02,  2.69862822e-02,  3.49039060e-02,
         1.53419217e-03, -1.10407363e-02,  5.57556285e-02, -2.88360142e-02,
        -4.87239498e-02, -9.90923923e-02,  6.02579764e-02, -1.86105379e-02,
        -8.81040139e-03, -3.01994831e-02, -1.42541815e-02, -3.87884212e-02,
        -2.76125662e-02, -6.94957533e-03,  3.20653319e-02, -5.17838394e-03,
         8.37959894e-02, -8.87329606e-02, -7.30260625e-02,  5.92846802e-02,
         4.07805674e-02,  7.72684822e-02, -4.60029697e-02,  3.98426895e-02,
         1.74821510e-02,  8.70263712e-03, -3.09291054e-02,  2.20470391e-02,
         4.76479896e-02,  1.76566196e-02, -3.62013627e-02, -3.53408062e-02,
        -5.92537995e-03,  2.97091202e-02,  8.05883752e-02,  1.07132722e-02,
        -5.38192519e-02, -9.62881111e-03, -1.17263739e-01,  3.36546374e-02,
         7.05763168e-03,  7.70159791e-03, -5.17482529e-02,  5.46955574e-02,
        -7.8

In [11]:
len(query_embedding)

1

In [12]:
vector1 = query_embedding[0]

In [13]:
min(vector1)

-0.11726373885183883

### Ans: The smallest value in the embedding is -0.11

### Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can chech that by using the norm function:

In [14]:
import numpy as np
np.linalg.norm(vector1)

1.0

### Q2. Cosine similarity with another vector

Now let's embed this document:

In [15]:
doc = 'Can I still join the course after the start date?'

In [16]:
doc_embedding = list(model.embed(doc))

In [17]:
doc_embedding

[array([-5.45303910e-02, -7.83451754e-02,  3.13610346e-02,  2.34234938e-02,
        -3.06321480e-02, -5.51041640e-02,  2.24726486e-02,  1.50386159e-02,
        -1.81159589e-02,  1.80595354e-02, -4.80892277e-03, -7.57395955e-02,
        -4.87272162e-03, -9.07717788e-02,  4.83940879e-02, -4.26317430e-02,
        -2.61462523e-02, -4.48129135e-02, -2.06926015e-02, -3.79054485e-02,
        -3.30329342e-02, -8.02850120e-03,  1.18560351e-02, -3.83721199e-02,
         7.26378149e-02, -5.82811752e-02, -7.30890167e-02,  3.40911381e-02,
         5.60705346e-02,  5.02684759e-02, -5.99293341e-02,  5.33016566e-02,
        -1.79524520e-02,  4.17745107e-03, -2.45427902e-02,  1.32288255e-02,
         4.64178315e-02,  1.09963765e-02, -1.01499639e-02, -5.77524144e-03,
        -2.83987757e-04,  4.91496354e-02,  1.11214836e-01, -2.76306424e-02,
        -6.78923985e-02, -1.85928261e-02, -1.23963205e-01,  1.25745922e-02,
         1.65298714e-02,  7.22242999e-03, -2.77237445e-02,  5.18073540e-02,
        -5.9

In [18]:
vector2 = doc_embedding[0]

In [19]:
cosine_similarity = vector1.dot(vector2)

In [20]:
cosine_similarity

0.9008528895674548

### Ans: the cosine similarity between the two vectors is 0.9

### Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

In [21]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):
Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication: V.dot(q)

In [44]:
text_fields = []
for doc in documents:
    text_fields.append(doc["text"])

In [45]:
text_embeddings_matrix = np.array(list(model.embed(text_fields)))
text_embeddings_matrix.shape

(5, 512)

In [47]:
similarity_vector = text_embeddings_matrix.dot(vector1)

In [48]:
similarity_vector

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

### Ans: index 1 has the highest similarity 

### Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

In [49]:
question_and_text_fields = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    question_and_text_fields.append(full_text)

In [50]:
question_and_text_embeddings_matrix = np.array(list(model.embed(question_and_text_fields)))
question_and_text_embeddings_matrix.shape

(5, 512)

In [51]:
similarity_vector = question_and_text_embeddings_matrix.dot(vector1)

In [52]:
similarity_vector

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

### Ans: Index 0 has the highest similarity, likely because the question is closer to the original query than the question at documents[1]

### Q5. Selecting the embedding model

In [55]:
supported_models = TextEmbedding.list_supported_models()
model_dims = [model["dim"] for model in supported_models]
model_dims.sort()
model_dims[0]

384

### Ans: The smallest dimensionality for models in fastembed is 384

### Q6. Indexing with qdrant (2 points)

In [57]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [58]:
model_handle = "BAAI/bge-small-en"
model = TextEmbedding(model_name=model_handle)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

C:\Users\llp_y\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\llp_y\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

In [68]:
# Define the collection name
collection_name = "llm-zoomcamp-hw2"
EMBEDDING_DIMENSIONALITY = 384

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

ResponseHandlingException: [WinError 10061] No connection could be made because the target machine actively refused it

In [59]:
question_and_text_fields = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    question_and_text_fields.append(full_text)

In [60]:
question_and_text_embeddings_matrix = np.array(list(model.embed(question_and_text_fields)))
question_and_text_embeddings_matrix.shape

(375, 384)

In [62]:
query_embedding = list(model.embed(query))

In [63]:
vector1 = query_embedding[0]